# Model Deployment on Vertex AI with Scikit-Learn

Contributors: michaelmenzel@google.com

In [9]:
!pip install -q --user --no-warn-conflicts google-cloud-aiplatform

In [10]:
from google.cloud import aiplatform

aiplatform.init(location='europe-west4')

## Model Training

In [11]:
%%writefile sklearn_train.py

from google.cloud import storage

import numpy as np

from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

import argparse
import joblib
import os

parser = argparse.ArgumentParser()
parser.add_argument('--gamma', dest='gamma',
                    default=0.001, type=float,
                    help='Gamma param for support vector classifier.')
args = parser.parse_args()

iris = datasets.load_iris()
X = iris.data
Y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.1, random_state=2184)
clf = svm.SVC(gamma=args.gamma)
clf.fit(X_train, y_train)

predicted = clf.predict(X_test)
print(metrics.classification_report(y_test, predicted, target_names=iris.target_names))

model_file = 'model.joblib'
gcs_path = os.getenv("AIP_MODEL_DIR")

joblib.dump(clf, model_file)

blob = storage.blob.Blob.from_string(os.path.join(gcs_path, model_file), client=storage.Client())
blob.upload_from_filename('model.joblib')

Overwriting sklearn_train.py


In [12]:
vertex_ai_job = aiplatform.CustomTrainingJob(
    display_name='sklearn-iris-classifier-training',
    script_path='sklearn_train.py',
    container_uri='europe-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest',
    staging_bucket='gs://sandbox-michael-menzel-training-europe-west4/sklearn-iris-classifier-training/',
    requirements=[],
    model_serving_container_image_uri='europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest',
)

vertex_ai_model = vertex_ai_job.run(
    replica_count=1, 
    args=[
        f"--gamma=0.001",
    ]
)

INFO:google.cloud.aiplatform.utils.source_utils:Training script copied to:
gs://sandbox-michael-menzel-training-europe-west4/sklearn-iris-classifier-training/aiplatform-2021-10-21-12:30:42.609-aiplatform_custom_trainer_script-0.1.tar.gz.
INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://sandbox-michael-menzel-training-europe-west4/sklearn-iris-classifier-training/aiplatform-custom-training-2021-10-21-12:30:42.717 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/914041608356233216?project=928871478446
INFO:google.cloud.aiplatform.training_jobs:CustomTrainingJob projects/928871478446/locations/europe-west4/trainingPipelines/914041608356233216 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:View backing custom job:
https://console.cloud.google.com/ai/platform/locations/europe-west4/training/5767796106754785280?project=928871478446
I

In [13]:
vertex_ai_endpoint = vertex_ai_model.deploy(
    deployed_model_display_name='sklearn-iris-classifier-model',
    traffic_split={"0": 100},
    machine_type="n1-standard-4",
    min_replica_count=1,
    max_replica_count=1,
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/928871478446/locations/europe-west4/endpoints/8903704424241692672/operations/501352013498417152
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/928871478446/locations/europe-west4/endpoints/8903704424241692672
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/928871478446/locations/europe-west4/endpoints/8903704424241692672')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/928871478446/locations/europe-west4/endpoints/8903704424241692672
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/928871478446/locations/europe-west4/endpoints/8903704424241692672/operations/5462067003047018496
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/9288714

In [15]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.1, random_state=2184)

vertex_ai_predicted = vertex_ai_endpoint.predict(X_test.tolist())
print(metrics.classification_report(y_test, vertex_ai_predicted.predictions, target_names=iris.target_names))

              precision    recall  f1-score   support

      setosa       0.43      1.00      0.60         3
  versicolor       0.50      0.33      0.40         6
   virginica       1.00      0.67      0.80         6

    accuracy                           0.60        15
   macro avg       0.64      0.67      0.60        15
weighted avg       0.69      0.60      0.60        15



In [16]:
vertex_ai_predicted

Prediction(predictions=[0.0, 2.0, 1.0, 0.0, 1.0, 2.0, 0.0, 1.0, 0.0, 2.0, 1.0, 0.0, 0.0, 2.0, 0.0], deployed_model_id='2118292713794174976', explanations=None)

## Cleanup

In [7]:
vertex_ai_endpoint.delete(force=True)

INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/928871478446/locations/europe-west4/endpoints/776958896651632640
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/928871478446/locations/europe-west4/endpoints/776958896651632640/operations/2176691074880241664
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/928871478446/locations/europe-west4/endpoints/776958896651632640
INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/928871478446/locations/europe-west4/endpoints/776958896651632640
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/928871478446/locations/europe-west4/operations/6788377093307629568
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/928871478446/locations/europe-west4/endpoints/776958896651632640


In [8]:
vertex_ai_model.delete()

INFO:google.cloud.aiplatform.base:Deleting Model : projects/928871478446/locations/europe-west4/models/4696427578603339776
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/928871478446/locations/europe-west4/operations/6927988681756114944
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/928871478446/locations/europe-west4/models/4696427578603339776
